In [1]:
import pyspark
from pyspark.sql import SparkSession, Row
from pyspark.sql.types import StructType, StructField, StringType, IntegerType

spark = SparkSession.builder.appName('Test').getOrCreate()

In [15]:
df = spark.read.options(delimiter=',').text('/home/benito.ramos/Desktop/GIT/PySpark/inputs/data_movies')

TypeError: csv() got an unexpected keyword argument 'delimiter'

In [14]:
df.show()

+------------------+
|               _c0|
+------------------+
|        HAN SOLO,4|
|        HAN SOLO,5|
|        HAN SOLO,6|
|        HAN SOLO,7|
|  LUKE SKYWALWER,4|
|  LUKE SKYWALWER,5|
|  LUKE SKYWALWER,6|
|  LUKE SKYWALWER,7|
|  LUKE SKYWALWER,8|
|ANAKIN SKYWALKER,1|
|ANAKIN SKYWALKER,2|
|ANAKIN SKYWALKER,3|
|     DARTH VADER,4|
|     DARTH VADER,5|
|     DARTH VADER,6|
|     DARTH VADER,7|
|     DARTH VADER,8|
|     DARTH VADER,9|
+------------------+

